In [ ]:
import logging
from noisepy.seis.io.compositerawstore import CompositeRawStore
from noisepy.seis.io.s3store import SCEDCS3DataStore, NCEDCS3DataStore
# from noisepy.seis.io.pnwstore import PNWDataStore
from noisepy.seis.io.channelcatalog import XMLStationChannelCatalog
from datetimerange import DateTimeRange
from datetime import datetime, timezone  
from noisepy.seis.io.channel_filter_store import channel_filter

logger = logging.getLogger("noisepy.seis.io")
logger.setLevel("DEBUG")
logger.addHandler(logging.StreamHandler())

In [ ]:
SCEDC_DATA = "s3://scedc-pds/continuous_waveforms/"
NCEDC_DATA = "s3://ncedc-pds/continuous_waveforms/NC/"
# PNW_DATA = "/1-fnp/pnwstore1/p-wd08/PNW2012/__/" 
# PNW_DB_PATH="/fd1/yiyu_data/PNWstore_sqlite/2012.sqlite"

S3_STORAGE_OPTIONS = {"s3": {"anon": True}}
SCEDC_STATION_XML = "s3://scedc-pds/FDSNstationXML/CI/"  
NCEDC_STATION_XML = "s3://ncedc-pds/FDSNstationXML/NC/"
# PNW_STATION_XML = "/1-fnp/pnwstore1/p-wd11/PNWStationXML/"

# timeframe for analysis
start = datetime(2012, 1, 1, tzinfo=timezone.utc)
end = datetime(2012, 1, 3, tzinfo=timezone.utc)
timerange = DateTimeRange(start, end)

In [ ]:
scedc_stations = "RPV,SVD,BBR".split(",")  # SCEDC station
ncedc_stations = "KCT,KRP,KHMB".split(",") # NCEDC station
# pnw_stations = "RATT,IRON,DBO".split(",") # UW station


scedc_catalog = XMLStationChannelCatalog(SCEDC_STATION_XML, 
                                         storage_options=S3_STORAGE_OPTIONS)
ncedc_catalog = XMLStationChannelCatalog(NCEDC_STATION_XML, "{network}.{name}.xml", 
                                         storage_options=S3_STORAGE_OPTIONS)
# pnw_catalog = XMLStationChannelCatalog(PNW_STATION_XML, "{network}/{network}.{name}.xml")


scedc_store = SCEDCS3DataStore(SCEDC_DATA, scedc_catalog,  channel_filter(["CI"], scedc_stations, ["BHE", "BHN", "BHZ"]), 
                               timerange, storage_options=S3_STORAGE_OPTIONS)
ncedc_store = NCEDCS3DataStore(NCEDC_DATA, ncedc_catalog, channel_filter(["NC"], ncedc_stations, ["HHE", "HHN", "HHZ"]), 
                               timerange, storage_options=S3_STORAGE_OPTIONS)
# pnw_store = PNWDataStore(PNW_DATA, pnw_catalog, PNW_DB_PATH, channel_filter(["UW", "UO"], pnw_stations, ["BHE", "BHN", "BHZ"]), 
                            #    timerange)

raw_store = CompositeRawStore({"CI": scedc_store, 
                               "NC": ncedc_store,})
# append for PNWstores
# "UW": pnw_store
# "UO": pnw_store

In [ ]:
ts = raw_store.get_timespans()
print(ts)
channels = raw_store.get_channels(ts[0])
print(channels)
data = raw_store.read_data(timerange, channels[0])
print(data.data.shape)